In [1]:
import pandas as pd
import numpy as np

In [2]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

In [3]:
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

In [4]:
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

In [5]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

In [6]:
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [7]:
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [8]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9658791 entries, 0 to 9658790
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 663.2+ MB


In [9]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35115304 entries, 0 to 35115303
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   titleId          object
 1   ordering         int64 
 2   title            object
 3   region           object
 4   language         object
 5   types            object
 6   attributes       object
 7   isOriginalTitle  object
dtypes: int64(1), object(7)
memory usage: 2.1+ GB


In [10]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1284591 entries, 0 to 1284590
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1284591 non-null  object 
 1   averageRating  1284591 non-null  float64
 2   numVotes       1284591 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 29.4+ MB


In [11]:
basics = basics.replace({'\\N':np.nan})

In [12]:
basics = basics.dropna(subset=['runtimeMinutes','genres'])

In [13]:
movie_filter = basics['titleType']=='movie'

In [14]:
basics = basics[movie_filter]

In [15]:
basics['startYear'] = basics['startYear'].astype(float)

In [16]:
startYear2000 = basics['startYear'] >=2000

In [17]:
startYear2022 = basics['startYear']<=2022

In [18]:
basics = basics[startYear2000 & startYear2022]

In [19]:
# Exclude movies that are included in the documentary category.
documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~documentary]

In [20]:
ratings = ratings.replace({'\\N':np.nan})

In [21]:
us_filter = akas['region']=='US'

In [22]:
akas = akas[us_filter]

In [23]:
akas = akas.replace({'\\N':np.nan})

In [24]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = basics['tconst'].isin(akas['titleId'])
keepers

34803       True
61116       True
67669       True
77964      False
86801       True
           ...  
9658464     True
9658473     True
9658512    False
9658557     True
9658641    False
Name: tconst, Length: 146714, dtype: bool

In [25]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama
...,...,...,...,...,...,...,...,...,...
9657929,tt9914942,movie,Life Without Sara Amat,La vida sense la Sara Amat,0,2019.0,NaN,74,Drama
9658324,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019.0,NaN,97,"Comedy,Drama,Fantasy"
9658464,tt9916170,movie,The Rehearsal,O Ensaio,0,2019.0,NaN,51,Drama
9658473,tt9916190,movie,Safeguard,Safeguard,0,2020.0,NaN,95,"Action,Adventure,Thriller"


In [26]:
keepers2 = ratings['tconst'].isin(akas['titleId'])
keepers2

0           True
1           True
2          False
3          False
4           True
           ...  
1284586    False
1284587    False
1284588    False
1284589    False
1284590    False
Name: tconst, Length: 1284591, dtype: bool

In [27]:
ratings = ratings[keepers2]
ratings

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
4,tt0000005,6.2,2593
5,tt0000006,5.1,177
6,tt0000007,5.4,812
...,...,...,...
1284556,tt9916200,8.2,221
1284557,tt9916204,8.2,253
1284564,tt9916348,8.3,18
1284565,tt9916362,6.4,5200


In [28]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85726 entries, 34803 to 9658557
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          85726 non-null  object 
 1   titleType       85726 non-null  object 
 2   primaryTitle    85726 non-null  object 
 3   originalTitle   85726 non-null  object 
 4   isAdult         85726 non-null  object 
 5   startYear       85726 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  85726 non-null  object 
 8   genres          85726 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.5+ MB


In [29]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34803,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61116,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67669,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86801,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93938,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [30]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1418401 entries, 5 to 35115048
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1418401 non-null  object
 1   ordering         1418401 non-null  int64 
 2   title            1418401 non-null  object
 3   region           1418401 non-null  object
 4   language         3840 non-null     object
 5   types            974792 non-null   object
 6   attributes       46099 non-null    object
 7   isOriginalTitle  1417056 non-null  object
dtypes: int64(1), object(7)
memory usage: 97.4+ MB


In [31]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490706 entries, 0 to 1284569
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         490706 non-null  object 
 1   averageRating  490706 non-null  float64
 2   numVotes       490706 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB


In [33]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
4,tt0000005,6.2,2593
5,tt0000006,5.1,177
6,tt0000007,5.4,812


In [34]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

In [35]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [36]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [37]:
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

In [38]:
akas = pd.read_csv("Data/title_akas.csv.gz", low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0


In [39]:
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [40]:
ratings = pd.read_csv("Data/title_ratings.csv.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1957
1,tt0000002,5.8,263
2,tt0000005,6.2,2593
3,tt0000006,5.1,177
4,tt0000007,5.4,812


In [41]:
# Install tmdbsimple (only need to run once)
#!pip install tmdbsimple

In [42]:
import json
with open('/Users/nusratjahan/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()

dict_keys(['API Key (v3 auth)'])

In [43]:
import tmdbsimple as tmdb
tmdb.API_KEY =  login['API Key (v3 auth)']

In [44]:
## make a movie object using the .Movies function from tmdb
movie = tmdb.Movies(603)

In [45]:
## movie objects have a .info dictionary 
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/waCRuAW5ocONRehP556vPexVXA9.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 76.465,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 372,
   'logo_path': None,
   'name': 'Groucho II Film

In [46]:
info['budget']

63000000

In [47]:
info['revenue']

463517383

In [48]:
info['imdb_id']

'tt0133093'

In [49]:
movie = tmdb.Movies('tt1361336')
info = movie.info()
info['budget']

50000000

In [50]:
# example from package README
# source = https://github.com/celiao/tmdbsimple
releases = movie.releases()
for c in releases['countries']:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

PG
PG
PG


In [51]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
    
    # save the .info .releases dictionaries
    info = movie.info()
    
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    return info

In [52]:
## testing our function by looping through a list of ids
test_ids = ["tt0848228", "tt0332280"]
results = []
errors = []
for movie_id in test_ids:
    
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
        
    except Exception as e: 
        errors.append([movie_id, e])
    
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.700,28252,PG-13
1,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115603229,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.878,10180,PG-13


In [53]:
print(f"- Number of errors: {len(errors)}")
errors

- Number of errors: 0


[]

In [54]:
import os, time,json
import tmdbsimple as tmdb 
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['title_basics.csv.gz', 'title_akas.csv.gz', 'title_ratings.csv.gz']

In [55]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [56]:
YEARS_TO_GET = [2000,2001]

In [57]:
errors = [ ]

In [71]:
#Defining the JSON file to store results for year
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

In [65]:
# Check if file exists
file_exists = os.path.isfile(JSON_FILE)

In [66]:
# If it does not exist: create it
if file_exists == False:
# save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)

In [67]:
#Saving new year as the current df
df = basics.loc[ basics['startYear']==YEAR].copy()
# saving movie ids to list
movie_ids = df['tconst'].copy()

In [68]:
# Load existing data from json into a dataframe called "previous_df"
previous_df = pd.read_json(JSON_FILE)

In [69]:
# filter out any ids that are already in the JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

In [73]:
from tqdm.notebook import tqdm_notebook

In [76]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    #Get index and movie id from list
    # INNER Loop
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        try:
            # Retrieve then data for the movie id
            temp = get_movie_with_rating(movie_id)  
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        except Exception as e:
            errors.append([movie_id, e])
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1434 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1434 [00:00<?, ?it/s]

In [75]:
print(f"- Total errors: {len(errors)}")

- Total errors: 424


In [ ]:
# Instead of previous_df=pd.read_json:
# previous_df = read_and_fix_json(JSON_FILE)